# Merge Ventilator data and HGHI State-level data

This notebook reads in the ventilator dataset and merges it with the state-level HGHI data.

From https://www.cambridge.org/core/journals/disaster-medicine-and-public-health-preparedness/article/mechanical-ventilators-in-us-acute-care-hospitals/F1FDBACA53531F2A150D6AD8E96F144D





In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
vents_path = '../data/ventilators_by_state.csv'
vents_df = pd.read_csv(vents_path, encoding='utf-8')

In [ ]:
hghi_state_gdf = gpd.read_file('../data/hghi_state_data.geojson', encoding='utf-8')

In [ ]:
merged_df = hghi_state_gdf.set_index('State').join(vents_df.set_index('State Abbrv')).reset_index()
merged_df = merged_df.drop(columns=['Location'])


In [ ]:
merged_gdf = gpd.GeoDataFrame(merged_df, crs=4326)

In [ ]:
merged_gdf.to_file('../data/hghi_state_data_with_vents.geojson', 
                  encoding='utf-8', 
                  driver='GeoJSON')